In [1]:
!pip install requests langchain google-generativeai python-dotenv

In [2]:
import os
import requests
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType

In [3]:
# Load environment variables
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [4]:
from unidecode import unidecode

# Step 1: Get latitude and longitude from IP
def get_location_by_ip():
    try:
        res = requests.get("http://ip-api.com/json/")
        data = res.json()
        lat = data.get("lat")
        lon = data.get("lon")
        city = unidecode(data.get("city", "Unknown"))
        return lat, lon, city
    except:
        return None, None, None


In [5]:
# Step 2: Get weather using coordinates
def get_weather_from_coordinates(_: str) -> str:
    lat, lon, city = get_location_by_ip()
    if lat is None or lon is None:
        return "❌ Could not detect your location."

    try:
        url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={OPENWEATHER_API_KEY}&units=metric"
        res = requests.get(url)
        data = res.json()

        if data.get("main"):
            desc = data["weather"][0]["description"]
            temp = data["main"]["temp"]
            humidity = data["main"]["humidity"]
            return (
                f"📍 Location: {city}\n"
                f"🌤️ Weather: {desc}\n"
                f"🌡️ Temp: {temp}°C\n"
                f"💧 Humidity: {humidity}%"
            )
        else:
            return "⚠️ Weather data not found."
    except Exception as e:
        return f"❌ Error: {str(e)}"

In [6]:
# Step 3: LangChain Tool
weather_tool = Tool(
    name="get_weather",
    func=get_weather_from_coordinates,
    description="Use this tool to get the current weather at the user's current location based on IP address."
)

In [7]:
# Step 4: Initialize Gemini LLM in LangChain
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    google_api_key=GOOGLE_API_KEY
)

agent = initialize_agent(
    tools=[weather_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


C:\Users\HP\AppData\Local\Temp\ipykernel_30404\844493991.py:8: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [8]:
# Step 5: Run query
query = "What's the weather like here?"
response = agent.run(query)

print("\n🌍 Final Weather Report:")
print(response)



> Entering new AgentExecutor chain...


C:\Users\HP\AppData\Local\Temp\ipykernel_30404\2448913814.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(query)


Thought: I need to get the current weather at the user's location to answer the question.
Action: get_weather
Action Input: ""
Observation: 📍 Location: Ludhiana
🌤️ Weather: clear sky
🌡️ Temp: 32.3°C
💧 Humidity: 71%
Thought:Thought: I now know the final answer
Final Answer: The weather in Ludhiana is clear sky with a temperature of 32.3°C and 71% humidity.

> Finished chain.

🌍 Final Weather Report:
The weather in Ludhiana is clear sky with a temperature of 32.3°C and 71% humidity.
